# Analysing ChIP-seq data with BioNumPy

In this task, we will use BioNumPy to analyse the transcription factor CTCF. We will work with a publically available dataset from the Encode Project.

The task is divided into four parts:

1) Filter/analyse the raw FASTQ reads
2) Look for motif matches in the FASTQ reads
3) Analyse the peaks
4) Analyse multiple peak data sets together (multiomics)


## Part 1